In [1]:
from langchain.document_loaders import WebBaseLoader

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:
html = WebBaseLoader("https://github.com/anishchapagain/anishchapagain.github.io/blob/master/README.md")
pages = html.load()

In [3]:
pages

[Document(page_content='\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nanishchapagain.github.io/README.md at master · anishchapagain/anishchapagain.github.io · GitHub\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSkip to content\n\n\n\n\n\n\n\n\n\n\n\n\nNavigation Menu\n\nToggle navigation\n\n\n\n\n\n\n\n\n\n \n\n\n\n\n\n\n\n\n            Sign in\n          \n\n\n\n\n\n\n\n\n        Product\n        \n\n\n\n\n\n\n\n\n\n\n\n\nActions\n        Automate any workflow\n      \n\n\n\n\n\n\n\nPackages\n        Host and manage packages\n      \n\n\n\n\n\n\n\nSecurity\n        Find and fix vulnerabilities\n      \n\n\n\n\n\n\n\nCodespaces\n        Instant dev environments\n      \n\n\n\n\n\n\n\nGitHub Copilot\n        Write better code with AI\n      \n\n\n\n\n\n\n\nCode review\n        Manage code c

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [5]:
separators=[
        "\n\n",
        "\n",
        " ",
        ".",
        ",",
        "\u200b",  # Zero-width space
        "\uff0c",  # Fullwidth comma
        "\u3001",  # Ideographic comma
        "\uff0e",  # Fullwidth full stop
        "\u3002",  # Ideographic full stop
        "",
    ]

In [6]:
r_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=5,
    separators=separators
)

In [7]:
docs = r_splitter.split_documents(pages)

In [8]:
docs

[Document(page_content='anishchapagain.github.io/README.md at master · anishchapagain/anishchapagain.github.io · GitHub', metadata={'source': 'https://github.com/anishchapagain/anishchapagain.github.io/blob/master/README.md', 'title': 'anishchapagain.github.io/README.md at master · anishchapagain/anishchapagain.github.io · GitHub', 'description': 'Anish Chapagain. Contribute to anishchapagain/anishchapagain.github.io development by creating an account on GitHub.', 'language': 'en'}),
 Document(page_content='Skip to content\n\n\n\n\n\n\n\n\n\n\n\n\nNavigation Menu', metadata={'source': 'https://github.com/anishchapagain/anishchapagain.github.io/blob/master/README.md', 'title': 'anishchapagain.github.io/README.md at master · anishchapagain/anishchapagain.github.io · GitHub', 'description': 'Anish Chapagain. Contribute to anishchapagain/anishchapagain.github.io development by creating an account on GitHub.', 'language': 'en'}),
 Document(page_content='Toggle navigation\n\n\n\n\n\n\n\n\n\n

In [9]:
len(docs)

50

In [10]:
docs[0].page_content

'anishchapagain.github.io/README.md at master · anishchapagain/anishchapagain.github.io · GitHub'

In [11]:
# Read API KEY from file
import json

file_path = 'llms_vault.json'
llm_key = 'GOOGLE_API_KEY'

try:
    with open(file_path,'r') as f:
        api_key = json.loads(f.read()).get(llm_key)
        if not api_key:
            raise ValueError(f"Some issue with '{llm_key}' from '{file_path}'. Verify the key:'{llm_key}' if it exists!")
except FileNotFoundError:
    raise FileNotFoundError(f"File related issue encountered, verify file path and name '{file_path}'")

# https://ai.google.dev/api
import google.generativeai as genai
genai.configure(api_key=api_key) # genai.configure(api_key=os.environ[llm_key]) 

In [12]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings # models/text-embedding-004	

import os
os.environ["GOOGLE_API_KEY"] = api_key

In [13]:
#result = genai.embed_content(model="models/text-embedding-004", content=docsr)

embedding_model = GoogleGenerativeAIEmbeddings(
    model="models/embedding-001", # text-embedding-004
    google_api_key=api_key,
    )

In [14]:
embedding_model

GoogleGenerativeAIEmbeddings(client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x000001BE40BCAFC0>, model='models/embedding-001', task_type=None, google_api_key=SecretStr('**********'), credentials=None, client_options=None, transport=None, request_options=None)

In [15]:
from langchain.vectorstores import Chroma

In [16]:
persist_directory = 'docs\\chromaE'

In [17]:
vectordb = Chroma.from_documents(
    documents=docs,
    embedding=embedding_model,
    persist_directory=persist_directory,
)

vectordb = Chroma.from_documents(
    documents=docsr,
    embedding=embeddings,
    persist_directory=persist_directory
)

In [18]:
vectordb

In [19]:
print(vectordb._collection.count())

50


In [20]:
googleDb = Chroma(
    persist_directory = persist_directory,
    embedding_function=embedding_model
    )
print("vector:",googleDb._collection.count(),"chunks.")

vector: 50 chunks.


In [21]:
googleDb

In [104]:
retriver = googleDb.as_retriever() 
# search_type="similarity",k=4)
# search_type="mmr"

In [68]:
docs_mmr = googleDb.max_marginal_relevance_search("list available URLs with text as provided in triple ticks '''github.io'''",k=3)
docs_mmr

[Document(page_content='Search or jump to...\n\n\n\n\n\n\n\nSearch code, repositories, users, issues, pull requests...', metadata={'description': 'Anish Chapagain. Contribute to anishchapagain/anishchapagain.github.io development by creating an account on GitHub.', 'language': 'en', 'source': 'https://github.com/anishchapagain/anishchapagain.github.io/blob/master/README.md', 'title': 'anishchapagain.github.io/README.md at master · anishchapagain/anishchapagain.github.io · GitHub'}),
 Document(page_content='Dismiss alert\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n        anishchapagain\n \n/\n\nanishchapagain.github.io\n\nPublic', metadata={'description': 'Anish Chapagain. Contribute to anishchapagain/anishchapagain.github.io development by creating an account on GitHub.', 'language': 'en', 'source': 'https://github.com/anishchapagain/anishchapagain.github.io/blob/master/README.md', 'title': 'anishchapagain.github.io/README.md at master · anishchapagain/anishchapagain.github.io · GitHub'}),

In [106]:
retriver

VectorStoreRetriever(tags=['Chroma', 'GoogleGenerativeAIEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x000001BE40D5C680>)

In [107]:
test = retriver.get_relevant_documents("list available URLs with text as provided in triple ticks '''github.io'''")
test

[Document(page_content='Search or jump to...\n\n\n\n\n\n\n\nSearch code, repositories, users, issues, pull requests...', metadata={'description': 'Anish Chapagain. Contribute to anishchapagain/anishchapagain.github.io development by creating an account on GitHub.', 'language': 'en', 'source': 'https://github.com/anishchapagain/anishchapagain.github.io/blob/master/README.md', 'title': 'anishchapagain.github.io/README.md at master · anishchapagain/anishchapagain.github.io · GitHub'}),
 Document(page_content='Insights\n\n\n\n\n\n \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nFooter\n\n\n\n\n\n\n\n\n        © 2024 GitHub,\xa0Inc.', metadata={'description': 'Anish Chapagain. Contribute to anishchapagain/anishchapagain.github.io development by creating an account on GitHub.', 'language': 'en', 'source': 'https://github.com/anishchapagain/anishchapagain.github.io/blob/master/README.md', 'title': 'anishchapagain.github.io/README.md at master · anishchapagain/anishchapagain.github.io · GitHub'}),
 Document

In [108]:
llm = retriver.invoke("list available URLs with text as provided in triple ticks '''github.io'''")

In [109]:
print(len(llm))

4


In [110]:
llm

[Document(page_content='Search or jump to...\n\n\n\n\n\n\n\nSearch code, repositories, users, issues, pull requests...', metadata={'description': 'Anish Chapagain. Contribute to anishchapagain/anishchapagain.github.io development by creating an account on GitHub.', 'language': 'en', 'source': 'https://github.com/anishchapagain/anishchapagain.github.io/blob/master/README.md', 'title': 'anishchapagain.github.io/README.md at master · anishchapagain/anishchapagain.github.io · GitHub'}),
 Document(page_content='Insights\n\n\n\n\n\n \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nFooter\n\n\n\n\n\n\n\n\n        © 2024 GitHub,\xa0Inc.', metadata={'description': 'Anish Chapagain. Contribute to anishchapagain/anishchapagain.github.io development by creating an account on GitHub.', 'language': 'en', 'source': 'https://github.com/anishchapagain/anishchapagain.github.io/blob/master/README.md', 'title': 'anishchapagain.github.io/README.md at master · anishchapagain/anishchapagain.github.io · GitHub'}),
 Document

In [57]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.schema import StrOutputParser

In [58]:
llm_model = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash-latest",
    google_api_key=api_key,
    )

In [59]:
from langchain import PromptTemplate

In [125]:
llm_prompt_template = """You are an assistant for question-answering tasks. Use the following context to answer the question.
If you don't know the answer, just say that you don't know. Use two to three sentences maximum and keep the answer concise.\n
Question: {question} \nContext: {context} \nAnswer:"""

llm_prompt_template = """You are an assistant, use the following context to answer the question.
Use two to three sentences maximum and keep the answer concise.\n
Question: {question} \nContext: {context} \nAnswer:"""


In [126]:
llm_prompt = PromptTemplate.from_template(llm_prompt_template)
print(llm_prompt)

input_variables=['context', 'question'] template='You are an assistant, use the following context to answer the question.\nUse two to three sentences maximum and keep the answer concise.\n\nQuestion: {question} \nContext: {context} \nAnswer:'


In [61]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [62]:
from langchain.schema.runnable import RunnablePassthrough

In [111]:
def reply(question=''):
    chain = (
    {"context": retriver | format_docs, "question": RunnablePassthrough()}
    | llm_prompt | llm_model | StrOutputParser()
    )
    print(chain.invoke(question))

In [145]:
reply("List name of some cities and country")

I'm sorry, but the provided context "Cancel" does not contain any information about cities or countries. Please provide a different context. 



In [144]:
reply("where is Himalayan Whitehouse College located")

The provided context does not include information about the location of Himalayan Whitehouse College. Therefore, I cannot answer your question. 



In [127]:
reply()

You are likely logged into the same account on multiple tabs or windows. To resolve this, refresh the current tab to sync your session and ensure you are using the correct account. 



In [112]:
reply("summarize the whole document in less that 200 words")

This document is a GitHub repository landing page for the "ReadME Project," which appears to focus on community articles. It includes sections for customer stories, partners, repositories, topics, and footer navigation. The page also provides links to terms, privacy, security, status, docs, and contact information. 



In [113]:
reply("provide me some technical roles mentioned")

The provided context mentions roles related to CI/CD & Automation, DevOps, and Security.  These roles likely include engineers, developers, and security analysts. 



In [115]:
reply("provide me full texts containing the word 'github.io'")

The provided context doesn't contain any text with the phrase "github.io." The only instance of "github.io" is in the URL "anishchapagain.github.io/README.md." 



In [116]:
reply("can you list me few places as where can i contribute to")

You can contribute by submitting feedback, opening issues, or creating pull requests.  The website encourages users to provide input and engage with the project. 



In [117]:
reply("what the content is tyring to say as 'Projects are life-less without'")

The content emphasizes that projects are inactive and lack purpose without actions.  They require concrete steps to be implemented and brought to life. 



In [129]:
reply("some quote by sadhguru is there, provide me the quote")

I cannot provide you with a quote from Sadhguru. I do not have access to the internet or external websites, including the one you provided. 



In [131]:
reply("what are the topics author is willing to get trained on")

The author is willing to be trained on DevSecOps, AI, code, issues, and pull requests. These topics are listed under the "Resources" section of the documentation. 



In [143]:
reply("list me URLs or links or email address from the page")

The provided context doesn't contain any URLs, links, or email addresses. It only shows a footer navigation with terms like Privacy, Security, Status, Docs, and Contact. 



# Tests

In [23]:
def search(question='') -> list:
    docs = googleDb.similarity_search(question, k=3)
    answers=[]
    for doc in docs:
        answers.append(doc.page_content[:200])
    
    return answers

In [26]:
search(question="list me some places where can i contribute to")

['Code\n\n\n\n\n\n\n\n\n\n\n          Issues\n\n\n\n\n\n\n\n\n\n\n          Pull requests',
 'anishchapagain.github.io/README.md at master · anishchapagain/anishchapagain.github.io · GitHub',
 'We read every piece of feedback, and take your input very seriously.']

In [28]:
search(question="list available URLs with text as provided in triple ticks '''github.io'''")

['Search or jump to...\n\n\n\n\n\n\n\nSearch code, repositories, users, issues, pull requests...',
 'Insights\n\n\n\n\n\n \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nFooter\n\n\n\n\n\n\n\n\n        © 2024 GitHub,\xa0Inc.',
 'Dismiss alert\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n        anishchapagain\n \n/\n\nanishchapagain.github.io\n\nPublic']

In [29]:
search(question="what is there mentioned as Willing to get trained in")

['Notifications\n You must be signed in to change notification settings\n\n\n \n\nFork\n    0',
 'Documentation\n\n    \n\n\n\n\n\n      GitHub Skills\n\n    \n\n\n\n\n\n      Blog',
 'anishchapagain.github.io/README.md at master · anishchapagain/anishchapagain.github.io · GitHub']

In [30]:
search("what we can work together in any topics")

['Discussions\n        Collaborate outside of code\n      \n\n\n\n\nExplore\n\n\n\n      All features',
 'Issues\n        Plan and track work',
 'Code\n\n\n\n\n\n\n\n\n\n\n          Issues\n\n\n\n\n\n\n\n\n\n\n          Pull requests']

In [32]:
search("list any text that's similar to honesty")

['Search syntax tips\n \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n        Provide feedback',
 'Footer navigation\n\n\nTerms\n\n\nPrivacy\n\n\nSecurity\n\n\nStatus\n\n\nDocs\n\n\nContact',
 'anishchapagain.github.io/README.md at master · anishchapagain/anishchapagain.github.io · GitHub']